In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import joblib

# 读取输入数据
input_file = "input.csv"  # 输入 CSV 文件
output_file = "output.csv"  # 输出 CSV 文件

df_input = pd.read_csv(input_file)
df_output = pd.read_csv(output_file)

# 选择输入特征
input_features = ["Time", "AntriebsmomentVA", "AntriebsmomentHA", "Lenkwinkel", "BremsmomentVA", "BremsmomentHA", "Geschw_S"]
output_features = ["Time", "KS_X", "KS_Y", "Gierwinkel", "Geschw_X", "Geschw_Y", "Giergeschwindigkeit"]

# 归一化数据
scalers = {}
for feature in input_features:
    scaler = MinMaxScaler()
    df_input[feature] = scaler.fit_transform(df_input[[feature]])
    scalers[feature] = scaler

for feature in output_features:
    scaler = MinMaxScaler()
    df_output[feature] = scaler.fit_transform(df_output[[feature]])
    scalers[feature] = scaler

# 保存归一化器
joblib.dump(scalers, "scalers.pkl")

# 创建时间序列数据集
def create_sequences(X, y, time_steps=10):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# 第一阶段：训练 Giergeschwindigkeit 预测模型
X1, y1 = create_sequences(df_input[input_features].values, df_output["Giergeschwindigkeit"].values)

model1 = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(X1.shape[1], X1.shape[2])),
    LSTM(50, activation='relu'),
    Dense(1)
])

model1.compile(optimizer='adam', loss='mse')
model1.fit(X1, y1, epochs=10, batch_size=32, verbose=1)
model1.save("model_giergeschwindigkeit.keras")

# 第二阶段：训练 Gierwinkel 预测模型
X2 = np.hstack([df_input[["Time", "Lenkwinkel"]].values[len(df_input) - len(y1):], y1.reshape(-1, 1)])
X2 = X2.reshape(X2.shape[0], 1, X2.shape[1])
y2 = df_output["Gierwinkel"].values[len(df_output) - len(X2):]

model2 = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(X2.shape[1], X2.shape[2])),
    LSTM(50, activation='relu'),
    Dense(1)
])

model2.compile(optimizer='adam', loss='mse')
model2.fit(X2, y2, epochs=10, batch_size=32, verbose=1)
model2.save("model_gierwinkel.keras")

# 第三阶段：训练 Geschw_X, Geschw_Y 预测模型
X3 = np.hstack([df_input[["Time", "Lenkwinkel", "Geschw_S"]].values[len(df_input) - len(y2):], y2.reshape(-1, 1)])
X3 = X3.reshape(X3.shape[0], 1, X3.shape[1])
y3 = df_output[["Geschw_X", "Geschw_Y"]].values[len(df_output) - len(X3):]

model3 = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(X3.shape[1], X3.shape[2])),
    LSTM(50, activation='relu'),
    Dense(2)
])

model3.compile(optimizer='adam', loss='mse')
model3.fit(X3, y3, epochs=10, batch_size=32, verbose=1)
model3.save("model_geschw.keras")

# 第四阶段：训练 KS_X, KS_Y 预测模型
X4 = np.hstack([df_input[["Time", "Lenkwinkel"]].values[len(df_input) - len(y3):], y3])
X4 = X4.reshape(X4.shape[0], 1, X4.shape[1])
y4 = df_output[["KS_X", "KS_Y"]].values[len(df_output) - len(X4):]

model4 = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(X4.shape[1], X4.shape[2])),
    LSTM(50, activation='relu'),
    Dense(2)
])

model4.compile(optimizer='adam', loss='mse')
model4.fit(X4, y4, epochs=10, batch_size=32, verbose=1)
model4.save("model_ks.keras")

print("所有模型训练完成，并已保存。")


Epoch 1/10


C:\Users\Zelin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


781/781 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0255
Epoch 2/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3.0912e-04
Epoch 3/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3.6463e-04
Epoch 4/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3.1054e-04
Epoch 5/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3.4901e-04
Epoch 6/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2.9600e-04
Epoch 7/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2.5630e-04
Epoch 8/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 2.9053e-04
Epoch 9/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2.6577e-04
Epoch 10/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2.5493e-04
Epoch 1/10


C:\Users\Zelin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1172
Epoch 2/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0648
Epoch 3/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0623
Epoch 4/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0576
Epoch 5/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0429
Epoch 6/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0135
Epoch 7/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0057
Epoch 8/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0042
Epoch 9/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0035
Epoch 10/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0032
Epoch 1/10


C:\Users\Zelin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.0607
Epoch 2/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0015
Epoch 3/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.6094e-04
Epoch 4/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.0438e-04
Epoch 5/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 7.9664e-05
Epoch 6/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6.5753e-05
Epoch 7/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 5.4903e-05
Epoch 8/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 4.7871e-05
Epoch 9/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 4.2342e-05
Epoch 10/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3.6642e-05
Epoch 1/10


C:\Users\Zelin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1036
Epoch 2/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 8.2806e-04
Epoch 3/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.5256e-04
Epoch 4/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.2491e-04
Epoch 5/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6.7532e-05
Epoch 6/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 5.0537e-05
Epoch 7/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 4.2687e-05
Epoch 8/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3.6965e-05
Epoch 9/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.7426e-05
Epoch 10/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.3171e-05
所有模型训练完成，并已保存。
